In [1]:
import pickle
import os
import pandas as pd
import numpy as np
from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [ ]:
# walk the experiment directory
experiments = {}
for root, dirs, files in os.walk('../experiments'):
    experiment = defaultdict(dict)
    for f in files:
        key = '/'.join(root.split('/')[1:-1])
        if f == "options.pkl":
            opts = vars(pickle.load(open(os.path.join(root, f), 'rb')))
            trial = opts['trial_number']
            experiment = experiments.get(key, defaultdict(dict))
            experiment[trial]['opts'] = opts
            experiments[key] = experiment
        elif f.startswith("event"):
            trial = root.split('/')[-1]
            experiment = experiments.get(key, defaultdict(dict))
            experiment[trial]['event_accumulator'] = EventAccumulator(os.path.join(root, f))
            experiments[key] = experiment
print experiments

{'experiments/experiments/percolate/cgn/testing123/b519c22480af0b72ef0d05910347f143': defaultdict(<type 'dict'>, {'1': {'event_accumulator': <tensorboard.backend.event_processing.event_accumulator.EventAccumulator object at 0x7fa4e992ed90>, 'opts': {'add_connectivity': False, 'nb_per_class': None, 'model_reg_lambda': 0.0, 'num_channel': 128, 'seed': 2, 'weight_decay': 0.0, 'extra_cn': 10, 'trial_number': '1', 'approx_nb_edges': 100, 'log': 'console', 'clinical_file': 'PANCAN_clinicalMatrix.gz', 'tensorboard_dir': './experiments/experiments/', 'extra_ucn': 0, 'epoch': 1, 'size_perc': 4, 'lr': 0.001, 'nb_attention_head': 0, 'train_ratio': 0.6, 'momentum': 0.9, 'use_emb': 16, 'attention_layer': 0, 'nb_nodes': None, 'load_folder': None, 'nb_examples': 100, 'dropout': False, 'center': False, 'batch_size': 100, 'pool_graph': 'hierarchy', 'percentile': 100, 'use_gate': 0.0, 'dataset': 'percolate', 'add_self': True, 'disconnected': 0, 'name': 'testing123', 'l1_loss_lambda': 0.0, 'scale_free': 

In [ ]:
trial = experiments.values()[0].values()[0]
scalar_tags = trial['event_accumulator'].Reload().Tags()['scalars']
opt_tags = trial['opts'].keys()
df = pd.DataFrame(columns=opt_tags + scalar_tags)
exp_df = pd.DataFrame(columns=opt_tags + scalar_tags)
for path, experiment in experiments.iteritems():
    for number, trial in experiment.iteritems():
        import pdb; pdb.set_trace()

        event_accumulator = trial['event_accumulator'].Reload()
        for tag in scalar_tags:
            events = event_accumulator.Scalars(tag)
            scalars = np.array(map(lambda x: x.value, events))
            df.loc[:, tag] = scalars
        for tag in opt_tags:
            df.loc[:, tag] = [trial['opts'][tag]]
        exp_df = exp_df.append(df)


> <ipython-input-3-015c6d574431>(10)<module>()
-> event_accumulator = trial['event_accumulator'].Reload()
(Pdb) n
> <ipython-input-3-015c6d574431>(11)<module>()
-> for tag in scalar_tags:
(Pdb) event_accumulator.__dict__
{'summary_metadata': {}, 'file_version': 2.0, '_tensor_summaries': {}, 'most_recent_step': -1, '_meta_graph': None, 'audios': <tensorboard.backend.event_processing.reservoir.Reservoir object at 0x7fa4e9946090>, 'accumulated_attrs': ('scalars', 'histograms', 'compressed_histograms', 'images', 'audios'), '_compression_bps': (0, 668, 1587, 3085, 5000, 6915, 8413, 9332, 10000), 'images': <tensorboard.backend.event_processing.reservoir.Reservoir object at 0x7fa4e9946050>, '_graph_from_metagraph': False, 'histograms': <tensorboard.backend.event_processing.reservoir.Reservoir object at 0x7fa4e992eed0>, 'compressed_histograms': <tensorboard.backend.event_processing.reservoir.Reservoir object at 0x7fa4e992ef90>, '_generator_mutex': <thread.lock object at 0x7fa4f05c5fd0>, '_gene

In [ ]:
for path, experiment in experiments.iteritems():
    for number, trial in experiment.iteritems():
        print trial